Recording the conversation. Press Enter to stop recording.


In [ ]:
import pyaudio
import wave
import threading

def record_audio(filename="conversation.wav"):
    chunk = 1024  
    format = pyaudio.paInt16  
    channels = 1  
    rate = 16000  
    frames = []

    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)
    
    print("Recording... Press Enter to stop recording.")
    
    def record():
        while not stop_event.is_set():
            data = stream.read(chunk)
            frames.append(data)

    stop_event = threading.Event()
    recorder_thread = threading.Thread(target=record)
    recorder_thread.start()
    
    # Wait for user input to stop recording
    input()  # Press Enter to stop
    stop_event.set()
    recorder_thread.join()

    print("Recording stopped.")
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recording
    wf = wave.open(filename, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

    print(f"Audio saved as {filename}")

# Run the function
record_audio()


Extract the text from the audio file


In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("conversation.wav",fp16=False)
print("Transcript:", result['text'])

Getting the summary of the conversation from gpt-4



In [ ]:
from openai import OpenAI

# Set up your API key (replace 'api-key' with your actual key)
client = OpenAI(api_key=api_key)
def summarize_conversation(transcript):
    prompt = f"""
    This is a doctor-patient conversation. Your task is to analyze the transcript and provide a summary of the interaction along with the following information:

    1. Symptoms: List all symptoms mentioned by the patient.
    2. Diagnosis: Mention the diagnosis made by the doctor, if available.
    3. Tests/Procedures: Identify any tests, procedures, or investigations advised by the doctor, if any.
    4. Medications: List any medications prescribed, if any.
    5. Follow-up: Mention if a follow-up or next appointment was suggested, if any.

    Here is the transcript of the conversation:

    {transcript}

    Please provide a detailed summary along with the extracted information.
    """

    # Call the GPT model
    response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for summarizing text. Provide complete, coherent summaries."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,  # Increased from 50 to 150
            temperature=0.2,
            presence_penalty=0.0,
            frequency_penalty=0.0,

        )
    summary = response.choices[0].message.content.strip()
    return summary

# Sample doctor-patient conversation
conversation = result['text']

# Get the summary and key points
summary = summarize_conversation(conversation)
print("Summary and Extracted Information:\n")
print(summary)
